In [114]:
########################CONSTANTS######################################
CBOW_N_WORDS = 4

MIN_WORD_FREQUENCY = 50
MAX_SEQUENCE_LENGTH = 256

EMBED_DIMENSION = 300
EMBED_MAX_NORM = 1

In [115]:
english_stopwords = set([
    'i', 'me', 'my', 'frequent','myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're",
    "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he',
    'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it',
    "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
    'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those',
    'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had',
    'having', 'do', 'does','sometimes', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if',
    'or', 'because', 'as','lastly', 'until', 'while', 'of', 'at', 'by', 'for', 'with',
    'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',
    'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over',
    'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where',
    'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other',
    'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too',
    'very', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now',
    'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn',
    "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn',
    "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't",
    'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn',
    "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn',
    "wouldn't"
])


In [116]:
################################DATALOADER##################################
import torch
from functools import partial
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import pandas as pd
from collections import Counter
import re
import os
from nltk.corpus import stopwords

"""from constants import
 (
    CBOW_N_WORDS,
    MIN_WORD_FREQUENCY,
    MAX_SEQUENCE_LENGTH,
)"""

class text_dataset(Dataset):
  def __init__(self, filepath, text_column="text"):
      df = pd.read_pickle(filepath)
      self.samples = df[text_column].tolist()

  def __getitem__(self, idx):
      return self.samples[idx]
  def __len__(self):
      return len(self.samples)


def data_iterator(data_dir, filename, text_column="text"):
    filepath = os.path.join(data_dir, filename)
    dataset = text_dataset(filepath, text_column)
    return dataset

def tokenizer(text:str):

  #stop_words = set(stopwords.words("english"))

  text = re.sub(r"</?s>|\[/?INST\]", "", text)
  text = re.sub(r"[^a-zA-Z]", " ", text)
  cleaned_text = text.lower()
  words = cleaned_text.split()
  words = [word for word in words if word not in english_stopwords]
  return words


def build_vocab(data_iter, tokenizer):

    sentences = map(tokenizer, data_iter)
    word_counts = Counter(word for sentence in sentences for word in sentence)
    vocab_words = [word for word, freq in word_counts.items() if freq >= MIN_WORD_FREQUENCY]
    #from word get ID
    word2id = {word: idx for idx, word in enumerate(vocab_words, start=1)}
    word2id["<unk>"] = 0
    #from ID get word
    id2word = {idx: word for word, idx in word2id.items()}
    return word2id,id2word

def collate_cbow(batch, text_pipeline):
  ##batch is a list of text paragraph
    batch_input, batch_output = [], []
    for text in batch:
      tokens_IDs = text_pipeline(text) # return a list of the words IDS
      if len(tokens_IDs) < CBOW_N_WORDS * 2 + 1:
          continue
      if MAX_SEQUENCE_LENGTH:
            tokens_IDs = tokens_IDs[:MAX_SEQUENCE_LENGTH]

      for idx in range(len(tokens_IDs) - CBOW_N_WORDS * 2):
            token_id_sequence = tokens_IDs[idx : (idx + CBOW_N_WORDS * 2 + 1)]
            output = token_id_sequence.pop(CBOW_N_WORDS)
            input_ = token_id_sequence
            batch_input.append(input_)
            batch_output.append(output)

    batch_input = torch.tensor(batch_input, dtype=torch.long)
    batch_output = torch.tensor(batch_output, dtype=torch.long)
    return batch_input, batch_output

def get_dataloader_word2id_id2word(filename,data_dir, batch_size, shuffle,word2id=None):

  data_itr   = data_iterator(data_dir, filename)
  tokenizer_ = tokenizer
  if not word2id:
    word2id,id2word = build_vocab(data_itr,tokenizer_)
  text_pipeline = lambda x: [word2id.get(word, 0) for word in tokenizer(x)]
  collate_fn = collate_cbow
  dataloader = DataLoader(
        data_itr,
        batch_size=batch_size,
        shuffle=shuffle,
        collate_fn=partial(collate_fn, text_pipeline=text_pipeline),
    )
  return dataloader, word2id, id2word

In [117]:
##########################MODEL#####################################
import torch.nn as nn
"""from constants import EMBED_DIMENSION, EMBED_MAX_NORM"""
class CBOW(nn.Module):
      def __init__(self, vocab_size: int):
        super(CBOW, self).__init__()
        self.Embedding = nn.Embedding(num_embeddings=vocab_size,embedding_dim=EMBED_DIMENSION,max_norm=EMBED_MAX_NORM)
        self.Outputs=nn.Linear(in_features=EMBED_DIMENSION,out_features=vocab_size)
      def forward(self,inputs):
        weights = self.Embedding(inputs)
        weights = weights.mean(axis=1)
        outputs = self.Outputs(weights)
        return outputs



In [118]:
#############################Helpers###############################################
import yaml
import torch
import torch.optim as optim
#from Utils.model import CBOW_Model

def get_model_class(model_name: str):
    if model_name == "CBOW":
        return CBOW
    else:
      raise ValueError("Only Available model is CBOW")
      return

def get_optimizer_class(optimizer:str):
   if optimizer == 'Adam':
        return optim.Adam
   else:
      raise ValueError("Only Available optimizer is Adam")
      return




In [119]:
############## Trainer #########################
import torch
import numpy as np
import torch.nn as nn


class Trainer:
    def __init__(self, model, epochs, train_dataloader, val_dataloader, train_steps, val_steps, optimizer,device):
        self.epochs = epochs                      # from config file
        self.train_dataloader = train_dataloader  # from class dataloader
        self.val_dataloader = val_dataloader      # from class dataloader
        self.train_steps = train_steps            # from config file
        self.val_steps = val_steps                # from config file
        self.model = model                        # create instance in train.py
        self.optimizer = optimizer                # create instance in train.py
        self.device = device                      #checked in the train.py

        self.criterion = nn.CrossEntropyLoss()    # fully defined
        self.model.to(self.device)                # adjust the model to the device
        self.loss = {"train": [], "val": []}

    def training(self):

      for epoch in range(self.epochs):
        self.train_epoch()
        self.val_epoch()
        print(f'Epoch:{epoch + 1}/{self.epochs},train_loss = {self.loss["train"][-1]:.4f} , val_loss = {self.loss["val"][-1]:.4f}')
      print('training has been completed :)')

    def train_epoch(self):

        running_loss = []
        self.model.train()

        for step , batch in enumerate (self.train_dataloader,start=1):

          #selecting batch & adjust to device
          inputs,targets  = batch
          inputs,targets  = inputs.to(self.device) , targets.to(self.device)

          #forward propagation
          self.optimizer.zero_grad()
          outputs_pred = self.model(inputs)
          loss  = self.criterion(outputs_pred,targets)

          #backward propagation
          loss.backward()
          self.optimizer.step()


          running_loss.append(loss.item())

          if step == self.train_steps:
            break

        epoch_loss = np.mean(running_loss)
        self.loss["train"].append(epoch_loss)

    def val_epoch(self):

        running_loss = []
        self.model.eval()

        with torch.inference_mode(): # turns off gradient tracking

          for step , batch in enumerate(self.val_dataloader,start=1):

            #selecting batch & adjust to device
            inputs,targets  = batch
            inputs,targets  = inputs.to(self.device) , targets.to(self.device)

            #forward propagation
            outputs_pred = self.model(inputs)
            loss  = self.criterion(outputs_pred,targets)
            running_loss.append(loss.item())

            if step == self.val_steps:
              break

          epoch_loss = np.mean(running_loss)
          self.loss["val"].append(epoch_loss)




In [120]:
####################Train################################
import yaml
import torch
import torch.nn as nn
#from utils.trainer import Trainer
"""from utils.helper import (
    get_model_class,
    get_optimizer_class,
    get_lr_scheduler,
    save_config,
    save_vocab,
)"""
#from Utils import get_dataloader_word2id_id2word

#reading config file
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)


#creating batches and get size of our vocab

train_dataloader, word2id , id2word = get_dataloader_word2id_id2word(filename=config['train_dataset'],
                                                               data_dir=config['data_dir'],
                                                               batch_size=config['train_batch_size'],
                                                               shuffle=config['shuffle'])
val_dataloader, _ , _ = get_dataloader_word2id_id2word(filename=config['val_dataset'],
                                                               data_dir=config['data_dir'],
                                                               batch_size=config['val_batch_size'],
                                                               shuffle=config['shuffle'])
vocab = len(word2id)

#create instances

model_class = get_model_class(config['model_name'])
model = model_class(vocab_size=vocab)

optimizer_class = get_optimizer_class(config['optimizer'])
optimizer = optimizer_class(params= model.parameters() ,lr=config['learning_rate'])

# select the deivce

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



# create instance of the trian class
trainer = Trainer(
    model = model,
    device = device,
    epochs = config['epochs'],
    train_dataloader = train_dataloader,
    val_dataloader   = val_dataloader,
    train_steps = config['train_steps'],
    val_steps   = config['val_steps'],
    optimizer= optimizer
)



In [121]:
trainer.training()

Epoch:1/50,train_loss = 6.0427 , val_loss = 5.8591
Epoch:2/50,train_loss = 5.7652 , val_loss = 5.4598
Epoch:3/50,train_loss = 5.3850 , val_loss = 5.0446
Epoch:4/50,train_loss = 5.0702 , val_loss = 4.7246
Epoch:5/50,train_loss = 4.8620 , val_loss = 4.4536
Epoch:6/50,train_loss = 4.7139 , val_loss = 4.2488
Epoch:7/50,train_loss = 4.6023 , val_loss = 4.0947
Epoch:8/50,train_loss = 4.5330 , val_loss = 3.9727
Epoch:9/50,train_loss = 4.4701 , val_loss = 3.9255
Epoch:10/50,train_loss = 4.4399 , val_loss = 3.8875
Epoch:11/50,train_loss = 4.3974 , val_loss = 3.8381
Epoch:12/50,train_loss = 4.3692 , val_loss = 3.8247
Epoch:13/50,train_loss = 4.3399 , val_loss = 3.8232
Epoch:14/50,train_loss = 4.3096 , val_loss = 3.8072
Epoch:15/50,train_loss = 4.2848 , val_loss = 3.8041
Epoch:16/50,train_loss = 4.2619 , val_loss = 3.7958
Epoch:17/50,train_loss = 4.2409 , val_loss = 3.7779
Epoch:18/50,train_loss = 4.2211 , val_loss = 3.7843
Epoch:19/50,train_loss = 4.1969 , val_loss = 3.7835
Epoch:20/50,train_los

In [124]:
import torch.nn.functional as F
model.Embedding


Embedding(463, 300, max_norm=1)

In [125]:
def get_vector(word):
    idx = word2id.get(word.lower(),0)
    if idx == 0:
        raise ValueError("OOV(out of vocabulary!!)")
        return
    return model.Embedding.weight[idx].detach()

def find_similar_words(word, top_n=10):
    vector = get_vector(word)
    if vector is None:
        return None
    all_vectors = model.Embedding.weight.detach()
    similarities = F.cosine_similarity(vector.unsqueeze(0), all_vectors)
    top_indices = torch.topk(similarities, top_n + 1).indices.tolist()

    results = []
    for i in top_indices:
        candidate = id2word[i]
        if candidate != word:
            similarity_score = similarities[i].item()
            similarity_percent = round(similarity_score * 100, 2)
            results.append((candidate, similarity_percent))
        if len(results) == top_n:
            break
    return results



In [126]:
v1 = get_vector('finger')
v2 = get_vector('leg')

In [127]:
find_similar_words('periods')

[('menstruation', 92.17),
 ('menstrual', 92.02),
 ('legs', 91.88),
 ('bleeding', 91.78),
 ('heavy', 90.67),
 ('hot', 89.93),
 ('low', 89.7),
 ('ache', 89.16),
 ('urination', 89.05),
 ('even', 88.96)]

In [128]:
find_similar_words('relieve')

[('help', 92.11),
 ('reduce', 88.88),
 ('use', 84.1),
 ('manage', 82.66),
 ('alleviate', 81.73),
 ('prevent', 81.57),
 ('used', 81.33),
 ('acid', 81.05),
 ('using', 80.35),
 ('topical', 80.19)]

In [129]:
find_similar_words('cardiac')

[('electrocardiogram', 93.35),
 ('check', 89.15),
 ('levels', 87.04),
 ('measure', 86.58),
 ('monitor', 84.56),
 ('intravenous', 83.75),
 ('panel', 83.42),
 ('test', 83.04),
 ('lipid', 82.81),
 ('rule', 82.71)]

In [130]:
find_similar_words('vomiting')

[('nausea', 96.45),
 ('sharp', 95.56),
 ('chest', 94.95),
 ('lower', 94.87),
 ('diarrhea', 94.8),
 ('heartburn', 94.65),
 ('abdominal', 93.71),
 ('fatigue', 93.71),
 ('upper', 93.02),
 ('burning', 92.89)]

In [112]:
torch.save(model, "word2vec_model.pth")

In [76]:
import pandas as pd


In [83]:
df = pd.DataFrame(word2id.items(), columns=['Word', 'ID'])


In [87]:
df_2 = pd.DataFrame(id2word.items(), columns=['ID','Word'])


In [91]:
df_2.to_pickle('id2word')

In [85]:
id2word

{1: 'doctor',
 2: 'experiencing',
 3: 'symptoms',
 4: 'lately',
 5: 'pain',
 6: 'hip',
 7: 'skin',
 8: 'rash',
 9: 'problems',
 10: 'face',
 11: 'could',
 12: 'causing',
 13: 'based',
 14: 's',
 15: 'possible',
 16: 'disease',
 17: 'time',
 18: 'recently',
 19: 'fever',
 20: 'shoulder',
 21: 'dizziness',
 22: 'eye',
 23: 'happening',
 24: 'may',
 25: 'syndrome',
 26: 'condition',
 27: 'body',
 28: 'like',
 29: 'examination',
 30: 'issues',
 31: 'noticed',
 32: 'blood',
 33: 'urine',
 34: 'vomiting',
 35: 'testicles',
 36: 'kidneys',
 37: 'causes',
 38: 'need',
 39: 'run',
 40: 'tests',
 41: 'confirm',
 42: 'diagnosis',
 43: 'trouble',
 44: 'feels',
 45: 'lungs',
 46: 't',
 47: 'properly',
 48: 'related',
 49: 'disorder',
 50: 'worried',
 51: 'high',
 52: 'pressure',
 53: 'take',
 54: 'check',
 55: 'series',
 56: 'medical',
 57: 'assess',
 58: 'hematologic',
 59: 'complete',
 60: 'count',
 61: 'cbc',
 62: 'lipid',
 63: 'panel',
 64: 'evaluate',
 65: 'levels',
 66: 'glucose',
 67: 'measu

In [310]:
import streamlit as st
import torch
import torch.nn.functional as F

# Assume you already have these loaded:
# - model
# - word2id
# - id2word

# --- Function to get the vector for a word
def get_vector(word):
    idx = word2id.get(word.lower(), 0)
    if idx is None or idx >= len(model.Embedding.weight):
        return None
    return model.Embedding.weight[idx].detach()

# --- Function to find similar words
def find_similar_words(word, top_n=5):
    vector = get_vector(word)
    if vector is None:
        return []
    all_vectors = model.Embedding.weight.detach()
    similarities = F.cosine_similarity(vector.unsqueeze(0), all_vectors)
    top_indices = torch.topk(similarities, top_n + 1).indices.tolist()
    similar_words = [
        (id2word[i], round(float(similarities[i].item()), 4))
        for i in top_indices if id2word[i] != word
    ][:top_n]
    return similar_words

# --- Streamlit UI
st.title("🔍 Word Similarity Explorer")
st.write("Enter a word to find semantically similar terms using your Word2Vec model.")

# Dropdown or input
selected_word = st.selectbox("Choose a word:", sorted(list(word2id.keys())))
top_n = st.slider("Number of similar words:", min_value=1, max_value=20, value=5)

if st.button("Find Similar Words"):
    similar = find_similar_words(selected_word, top_n=top_n)
    if similar:
        st.success("Top similar words:")
        for w, score in similar:
            st.write(f"**{w}** — Similarity: `{score * 100:.2f}%`")
    else:
        st.warning("Word not found or no similar words available.")


2025-07-03 07:27:01.528 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-03 07:27:01.530 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-03 07:27:01.531 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-03 07:27:01.532 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-03 07:27:01.534 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-03 07:27:01.537 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-03 07:27:01.538 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-03 07:27:01.539 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [113]:
torch.save(model.state_dict(), "cbow_weights.pth")